In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
## 设置字体
from matplotlib.font_manager import FontProperties
import pandas as pd
fonts = FontProperties(fname = "/Library/Fonts/华文细黑.ttf",size=14)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [145]:
train = pd.read_csv("../data/train.csv")
train = train.dropna()
train.head()

,content,label
0,什么是电气一次设备和一次回路,1
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0
3,什么是电气二次设备和二次回路,1
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0


In [146]:
train.shape,train[train['label'] == 1].shape

((5912, 2), (1593, 2))

In [147]:
def filter_abnormal(data):
    if len(data.content) > 30 or data.label==1:
        return data
new_train = train.apply(lambda x:filter_abnormal(x),axis=1).dropna()
new_train.shape

(3539, 2)

In [148]:
new_train.shape,new_train[train['label'] == 1].shape

D:\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


((3539, 2), (1593, 2))

In [182]:
new_train = train.copy()

In [183]:
from LAC import LAC
lac = LAC(mode='lac')
with open("../data/stopwords.txt",'rb') as f:
    stop_words = f.read().decode("utf-8")
stop_words = stop_words.split("\r\n")
def cut_words(sentence):
    words_list = []
    words,typ = lac.run(sentence)
    for i in range(len(words)):
#         if typ[i] != "n" and typ[i] != 'm':
#         if typ[i] != "n" and typ[i] != 'm' and words[i] not in stop_words:
        words_list.append(words[i])
    return " ".join(words_list)
new_train['tokens'] = new_train.content.apply(lambda x: cut_words(x))

In [184]:
new_train

,content,label,tokens
0,什么是电气一次设备和一次回路,1,什么 是 电气 一次 设备 和 一次 回路
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0,一次 设备 是 指 直接 生产 输送 和 分配 电能 的 高压 电气设备 它 包括 发电机变...
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0,由 一次 设备 相互 连接 构成 发电 输电 配电 或 进行 其它 生产 的 电气回路 称为...
3,什么是电气二次设备和二次回路,1,什么 是 电气 二次 设备 和 二次 回路
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0,二次 设备 是 指 对 一次 设备 的 工作 进行 监测 控制 调节 保护 以及 为 运行 ...
...,...,...,...
7207,气体的绝缘介质强度与气体压力具有较大的关系只有使气压保,0,气体 的 绝缘 介质 强度 与 气体 压力 具有 较大 的 关系 只有 使 气压 保
7208,持在设计规定的范围之内避免泄漏才能使电气设备达到或维持在设计的绝缘水,0,持 在 设计 规定 的 范围 之 内 避免 泄漏 才能 使 电气设备 达到 或 维持 在 设...
7209,平上运行,0,平上运行
7210,防毒面具是什么？,1,防毒面具 是 什么 ？


In [185]:
train_y = new_train.label.astype("int").values
new_train.head()

,content,label,tokens
0,什么是电气一次设备和一次回路,1,什么 是 电气 一次 设备 和 一次 回路
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0,一次 设备 是 指 直接 生产 输送 和 分配 电能 的 高压 电气设备 它 包括 发电机变...
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0,由 一次 设备 相互 连接 构成 发电 输电 配电 或 进行 其它 生产 的 电气回路 称为...
3,什么是电气二次设备和二次回路,1,什么 是 电气 二次 设备 和 二次 回路
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0,二次 设备 是 指 对 一次 设备 的 工作 进行 监测 控制 调节 保护 以及 为 运行 ...


In [186]:
le = LabelEncoder()
train_y = le.fit_transform(new_train.label).reshape(-1,1)
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
train_y

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [187]:
## 使用Tokenizer对词组进行编码
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 10000
max_len = 10
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(new_train.tokens)

## 使用word_index属性可以看到每次词对应的编码
## 使用word_counts属性可以看到每个词对应的频数
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("= = = = = =")
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('的', 1)
('在', 2)
('是', 3)
('？', 4)
('和', 5)
('时', 6)
('有', 7)
('中', 8)
('树脂', 9)
('工作', 10)
= = = = = =
('什么', 475)
('是', 1460)
('电气', 97)
('一次', 79)
('设备', 567)
('和', 1240)
('回路', 149)
('指', 83)
('直接', 57)
('生产', 118)


In [188]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(new_train.tokens)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
train_seq_mat.shape

(5912, 10)

In [189]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_seq_mat,train_y, random_state=2,train_size=0.8)

In [190]:
X_train.shape

(4729, 10)

In [191]:
X_test.shape

(1183, 10)

In [192]:
model = Sequential()
model.add(Embedding(max_words+1,60,input_length=max_len))
def build_model(num_layers,hidden_size,max_words=max_words,max_len=max_len,num_classes=2):
        '''
        hidden_size：LSTM隐藏层
        return_sequences:Boolen,return_sequences = True，
        则RNN层还可以返回每个样本的整个输出序列（每个样本每个时间步一个向量）。
        该输出的形状是（batch_size，timesteps，units）
        input_shape:输入训练数据的维度
        '''
        model = Sequential([
            Embedding(max_words+1,60,input_length=max_len),
            LSTM(hidden_size, return_sequences=True),
        ])
        for i in range(2,num_layers+1):
            if i == num_layers:
                model.add(LSTM(hidden_size, return_sequences=False))
            else:
                model.add(LSTM(hidden_size, return_sequences=True))
        model.add(Dense(128,activation="relu",name="FC1"))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes,activation="sigmoid",name="FC2"))
        return model

In [193]:
model = build_model(3,64)
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 10, 60)            600060    
_________________________________________________________________
lstm_18 (LSTM)               (None, 10, 64)            32000     
_________________________________________________________________
lstm_19 (LSTM)               (None, 10, 64)            33024     
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                33024     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               8320      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 2)               

In [194]:
model.compile(optimizer=Adam(0.001),
        loss='binary_crossentropy',
        metrics=['accuracy'])

In [195]:
# logdir = "..\logs\\" + datetime.datetime.now().strftime("%Y%m%d")

# if not os.path.exists(logdir):
#     os.mkdir(logdir)
# output_model_file = os.path.join(logdir,
#                                  "fashion_mnist_model.h5")

# callbacks = [
#     tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, profile_batch = 100000000),
#     tf.keras.callbacks.ModelCheckpoint(output_model_file,
#                                     save_best_only = True),
#     tf.keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
# ]
# history = model.fit(x_train_scaled, y_train, epochs=10,
#                     validation_data=(x_valid_scaled, y_valid),
#                     callbacks = callbacks)

In [196]:
# import datetime
# import tensorflow as tf
# import os
# log_dir = "..\logs\\" + datetime.datetime.now().strftime("%Y%m%d")
# folder = os.path.exists(log_dir)
# if not folder:
#     os.makedirs(log_dir)
# callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, profile_batch = 100000000)]

In [197]:
model_fit = model.fit(X_train,y_train,batch_size=1024,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
5/5 [==============================] - 1s 134ms/step - loss: 0.6871 - accuracy: 0.7056 - val_loss: 0.6745 - val_accuracy: 0.7194
Epoch 2/100
5/5 [==============================] - 0s 18ms/step - loss: 0.6584 - accuracy: 0.7333 - val_loss: 0.6266 - val_accuracy: 0.7194
Epoch 3/100
5/5 [==============================] - 0s 18ms/step - loss: 0.5988 - accuracy: 0.7333 - val_loss: 0.6058 - val_accuracy: 0.7194
Epoch 4/100
5/5 [==============================] - 0s 17ms/step - loss: 0.5842 - accuracy: 0.7333 - val_loss: 0.5790 - val_accuracy: 0.7194
Epoch 5/100
5/5 [==============================] - 0s 18ms/step - loss: 0.5653 - accuracy: 0.7333 - val_loss: 0.5726 - val_accuracy: 0.7194
Epoch 6/100
5/5 [==============================] - 0s 18ms/step - loss: 0.5492 - accuracy: 0.7333 - val_loss: 0.5482 - val_accuracy: 0.7194
Epoch 7/100
5/5 [==============================] - 0s 18ms/step - loss: 0.5043 - accuracy: 0.7333 - val_loss: 0.5047 - val_accuracy: 0.7227
Epoch 8/100
5/5 [==

5/5 [==============================] - 0s 18ms/step - loss: 0.0097 - accuracy: 0.9960 - val_loss: 0.9609 - val_accuracy: 0.8867
Epoch 60/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0126 - accuracy: 0.9964 - val_loss: 0.9573 - val_accuracy: 0.8791
Epoch 61/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0109 - accuracy: 0.9960 - val_loss: 0.9300 - val_accuracy: 0.8842
Epoch 62/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0109 - accuracy: 0.9966 - val_loss: 0.9001 - val_accuracy: 0.8901
Epoch 63/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0104 - accuracy: 0.9962 - val_loss: 0.8912 - val_accuracy: 0.8724
Epoch 64/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0091 - accuracy: 0.9968 - val_loss: 0.9054 - val_accuracy: 0.8859
Epoch 65/100
5/5 [==============================] - 0s 18ms/step - loss: 0.0110 - accuracy: 0.9964 - val_loss: 0.9196 - val_accuracy: 0.8850
Epoch 66/100
5/5 [========

In [198]:
model_fit = model.fit(train_seq_mat,train_y,batch_size=1024,epochs=300)

Epoch 1/300
6/6 [==============================] - 0s 13ms/step - loss: 0.1706 - accuracy: 0.9731
Epoch 2/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0647 - accuracy: 0.9827
Epoch 3/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0505 - accuracy: 0.9865
Epoch 4/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0413 - accuracy: 0.9888
Epoch 5/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0305 - accuracy: 0.9910
Epoch 6/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0246 - accuracy: 0.9924
Epoch 7/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0211 - accuracy: 0.9934
Epoch 8/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0173 - accuracy: 0.9937
Epoch 9/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0164 - accuracy: 0.9944
Epoch 10/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0153 - accuracy: 0.9944
Epoch 11/300
6/6 [=

6/6 [==============================] - 0s 12ms/step - loss: 0.0070 - accuracy: 0.9953
Epoch 84/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0070 - accuracy: 0.9949
Epoch 85/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0068 - accuracy: 0.9959
Epoch 86/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0072 - accuracy: 0.9944
Epoch 87/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0064 - accuracy: 0.9964
Epoch 88/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0070 - accuracy: 0.9954
Epoch 89/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0071 - accuracy: 0.9954
Epoch 90/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9954
Epoch 91/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0070 - accuracy: 0.9951
Epoch 92/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0068 - accuracy: 0.9951
Epoch 93/300
6/6 [=====

6/6 [==============================] - 0s 13ms/step - loss: 0.0067 - accuracy: 0.9953
Epoch 164/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0070 - accuracy: 0.9956
Epoch 165/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9953
Epoch 166/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0066 - accuracy: 0.9956
Epoch 167/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0065 - accuracy: 0.9959
Epoch 168/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0066 - accuracy: 0.9956
Epoch 169/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0068 - accuracy: 0.9946
Epoch 170/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0068 - accuracy: 0.9958
Epoch 171/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9954
Epoch 172/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9956
Epoch 173/300


6/6 [==============================] - 0s 13ms/step - loss: 0.0066 - accuracy: 0.9954
Epoch 245/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0065 - accuracy: 0.9958
Epoch 246/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0066 - accuracy: 0.9954
Epoch 247/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0066 - accuracy: 0.9946
Epoch 248/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0066 - accuracy: 0.9949
Epoch 249/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0067 - accuracy: 0.9944
Epoch 250/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9944
Epoch 251/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0066 - accuracy: 0.9954
Epoch 252/300
6/6 [==============================] - 0s 12ms/step - loss: 0.0066 - accuracy: 0.9953
Epoch 253/300
6/6 [==============================] - 0s 13ms/step - loss: 0.0066 - accuracy: 0.9959
Epoch 254/300


In [223]:
model.save('../model/lstm_model.h5')

In [250]:
def predict_ques(sentence):
    test_data = cut_words(sentence)
    val_seq = tok.texts_to_sequences([test_data])
    ## 将每个序列调整为相同的长度
    val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)
    ## 对验证集进行预测
    val_pre = model.predict(val_seq_mat)
    if val_pre.argsort()[0][1] == 1:
        return True
    else:
        return False

In [260]:
with open("C:/Users/melon/Downloads/ask题库拆解0605共891个/ask题库拆解0605共891个/问答30/test1.txt","rb") as f:
    test_text = f.read().decode("utf-8")

In [256]:
test_text.split(" ")

['基层神经外科医生理论问答',
 '神',
 '经',
 '外',
 '科',
 '',
 '简述正常和高颅压情况下颅内容积和代偿调节机制',
 '颅腔是一不能伸缩的容器，总体积固定不变。',
 '腔内脑组织、脑脊液、血液三者的总体积与颅腔容积相适应。三种内容物中，脑组织不能伸缩，容积代偿作用最小。脑脊液和血液是流动的，对颅腔代偿起重要作用。',
 '正常颅腔容积代偿为8%~10%，颅腔内三种内容物中任何一种体积的增大，其他两种内容物同时或至少一种必然相应地代偿性缩减，以取得平衡，使颅内压维持在正常范围内，这就是颅内压的生理调节。',
 '颅内压增高时，主要靠脑脊液吸收增加，分泌减少和被排挤入脊蛛网膜下腔，其次靠颅内静脉系统血液被排挤出颅外和脑血管收缩，使脑血流量减少来调节。',
 '简述颅内压增高的病理因素',
 '1、颅脑外伤后脑水肿是因毛细血管通透性改变，血管内液体外渗所致。',
 '2、颅内占位性病变，如颅内肿瘤、血肿、脓肿、结核瘤、寄生虫、囊肿等占据了颅内部分空间。',
 '3、颅内大血管急性梗塞伴发脑水肿。',
 '4、脑与脑膜炎症。',
 '5、颅内大静脉受压或栓塞所致的颅内血液回流障碍。',
 '6、脑脊液循环通路受阻，脑脊液产生过多或吸收障碍。',
 '7、全身性疾病，如中毒、休克及电解质紊乱等所引起的脑水肿。',
 '简述脑水肿的发生机制',
 '1、血脑屏障功能障碍',
 '',
 '是血管源性脑水肿的发病基础，是因毛细血管通透性增加，血浆外溢、聚集于脑细胞外间隙而造成。',
 '',
 '2、脑细胞代谢障碍',
 '',
 '各种原因引起的脑细胞缺氧缺血，细胞内ATP减少，钠泵不能正常工作，细胞内NaCL增多，水分大量进入细胞内，那么导致细胞毒性脑水肿。',
 '',
 '3、脑脊液循环障碍',
 '',
 '是形成间质性脑水肿的基础。各种脑积水时，脑室内脑脊液因压力高通过受损的室管膜进入脑室周围的白质间隙。',
 '',
 '4、血浆渗透压降低',
 '',
 '急性水中毒、低钠血症时，细胞外液呈低渗透压状态，液体进入细胞内，则细胞肿胀。',
 '',
 '5、流体静力压变化的影响',
 '',
 '严重或快速的动脉压增高或静脉回流受阻，导致毛细血管床压力增高，可致细胞外脑水肿。',
 '何谓脑疝，简介其种类',
 '',
 '',
 '

In [261]:
def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False
def format_str(content):
    content_str = ''
    for i in content:
        if is_chinese(i) or i=='?' or i=='？':
            content_str = content_str+i
    return content_str

In [271]:
question = []
pred = []
for item in test_text.split("\n"):
    if item:
        if predict_ques(format_str(item)):
                question.append(item)
                pred.append(1)
        else:
            pred.append(0)

In [265]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

In [268]:
content = []
label = []
for item in test_text.split("\n"):
    if item:
        if is_number(item.replace(" ","").split("、")[0]):
            content.append(item)
            label.append(1)
        else:
            content.append(item)
            label.append(0)

In [272]:
len(pred),len(label)

(1566, 1566)

In [275]:
from sklearn import metrics
metrics.accuracy_score(label,pred)

0.8167305236270753